In [212]:
#Import packages
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import shutil
import time
import win32com.client

In [213]:
#Todays date
TodaysDate = time.strftime("%#m.%d.%Y")

In [214]:
#Copperleaf username and password
username = 'vorbeckj'
password = '#'

In [215]:
#Copperleaf region codes
regions = ['bos','col','kedli','kedny','ri','ups']

In [216]:
#Read in list of investments
investments = pd.read_excel('C:/users/vorbej1/desktop/nyc_backup.xlsx',sheet_name='KEDNY', skiprows=3)

In [217]:
#Drop NAs - excel formatting issue
investments = investments.dropna(subset=['Investment Name']) 

In [218]:
#Initalize dataframe
backup_check = pd.DataFrame()
backup_check['Investments'] = investments['Investment Name']
backup_check['yes/no'] = ''

In [219]:
#Load Webdriver
driver = webdriver.Chrome('C:/Users/vorbej1/desktop/chromedriver.exe')

In [220]:
#Load C55
driver.get('https://ngg-prod.copperleafgroup.com/NGG_PROD/Copperleaf5/Pages/Home/Views/Home.aspx')

In [221]:
#Send username and password
driver.find_element_by_id('username').send_keys(username)
driver.find_element_by_id('password').send_keys(password)

In [222]:
#Submit
driver.find_element_by_xpath('//input[@tabindex="4"]').click()
time.sleep(5)

In [223]:
#Select investment search
driver.find_element_by_xpath('//a[@title="Search for investments using the advanced search features"]').click()
time.sleep(5)

In [224]:
#Remove filter if applied       
try:
    driver.find_element_by_class_name("cl-master__search-button--clear-filter").click()
except:
    pass

time.sleep(1.5)

In [225]:
#Locate filter button and apply filter to region in focus
driver.find_elements_by_xpath('//a[@tabindex="-1"]')[2].click()
time.sleep(2)

region_filter = driver.find_element_by_xpath('//li[@class="k-item k-filter-item k-state-default"]')
hover = ActionChains(driver).move_to_element(region_filter)
hover.perform()

filter_form = driver.find_element_by_xpath('//input[@title="Value"]')
time.sleep(1)

filter_form.send_keys(regions[3])
filter_form.send_keys(Keys.RETURN)

In [226]:
#For each investment in the excel file, search the database
#Depending on if investment has its backup data, assign yes/no
#Assign n/a to investments not in the database
for i in backup_check.index:
    #print(i)
    search_bar = driver.find_element_by_class_name('k-input')
    search_bar.send_keys(backup_check.loc[i,'Investments'])
    search_bar.send_keys(Keys.RETURN)
    time.sleep(.5)
    try:
        driver.find_element_by_link_text(backup_check.loc[i,'Investments']).click()
        time.sleep(1)
        driver.find_element_by_xpath('//span[@class="k-sprite k-icon cl-k__icon--attachment"]').click()
        time.sleep(1.5)
        if driver.find_elements_by_xpath("//span[contains(text(), 'Backup')]") and driver.find_elements_by_xpath("//span[contains(text(), '2019')]"):
            backup_check.loc[i, 'yes/no'] = 'yes'
        else:
            backup_check.loc[i, 'yes/no'] = 'no'
        driver.execute_script("window.history.go(-1)")
        time.sleep(2)
    except:
        time.sleep(1)
        backup_check.loc[i, 'yes/no'] = 'n/a'
        search_bar.clear()

In [227]:
#Write dataframe to csv for emailer
backup_check.to_csv('C:/Users/vorbej1/desktop/backup_check.csv', sep=',')

In [229]:
#Emailer
outlook = win32com.client.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = '#'
mail.Subject = 'C55 Backup Checks ' + TodaysDate  

attachment = mail.Attachments.Add('C:/Users/vorbej1/desktop/backup_check.csv')

mail.HTMLBody = "\<html>\
<head>\
<meta http-equiv=""Content-Type"" content=""text/html; charset=iso-8859-1"">\
\
<style type=""text/css"">\
<!--\
\
body { font-size: 20pt font-family: sans-serif; }\
\
-->\
</style>\
</head>\
<body>\
    Hello\
</body>\
</html>"

mail.Send()

print('Email Complete')